In [ ]:
import pickle
with open('subset.pickle','rb') as h:
    data = pickle.load(h)

### Grouping specialties

In [ ]:
group_specialties = {
 'internist-geriatrician':'int',
 'gastroenterologist':'gas',
 'family-gp':'family',
 'pain-mgmt-physical-rehab':'bone',
 'reproductive-endocrinologist':'gyn',
 'orthopedics-sports':'bone',
 'dermatologist':'skin',
 #'acupuncturist':'alt',
 #'naturopath':'alt',
 'ophthalmologist':'eye',
 'psychiatrist':'psych',
 'cosmetic-plastic-surgeon':'skin',
 #'optometrist':'eye',
 #'chiropractor':'bone',
 'pediatrician':'pedia',
 'neurosurgeon':'brain',
 #'orthodontist':'dent',
 'endocrinologist':'endo',
 'neurologist':'brain',
 #'dentist':'dent',
 #'podiatrist':'bone',
 'surgeon-general':'surg',
 'ear-nose-and-throat-ent':'ent',
 'gynecologist-obgyn':'gyn',
 #'psychologist':'psych',
 #'oral-surgeon':'dent'
}

for i in range(len(data)):
    entry = data.iloc[i]
    name = entry.name
    try:
        data.set_value(name,'specialty', group_specialties[entry['specialty']])
    except:
        data.set_value(name,'specialty', '-1')
    if i%1000 == 0:
        print(i)

In [ ]:
import pickle
#with open('grouped.pickle','wb') as h:
#    pickle.dump(data,h,protocol=2)
with open('grouped.pickle','rb') as h:
    data = pickle.load(h)

In [ ]:
data.iloc[10]
df = data.groupby('doc_name').apply(lambda x: x.sum())
df.iloc[1]
df.iloc[1]['text']

In [ ]:
# also make a set of specialties and genders
specialties, genders = set(), set()
# make a dict of specialty and gender by doc name
d = {}
for i in range(len(data)):
    doc = data.iloc[i]
    s = doc['specialty']
    g = doc['gender']
    specialties.add(s)
    genders.add(g)
    d[doc['doc_name']] = [s,g]

In [ ]:
specialties = list(specialties)
specialties.remove('-1')
specialties.append('-1') # HAHA! Now we've got -1 at the end, as default.
s_map = {}
for i,s in enumerate(specialties):
    s_map[s] = i
genders = list(genders)
print(len(specialties),len(genders))
genders
specialties

### Maintain vocabulary of required words.

In [ ]:
V_freq = {}
    
for i in range(len(df)):
    #if d[df.iloc[i].name][0] == '-1':
    #   continue
    for w in df.iloc[i]['text']:
        try:
            V_freq[w] += 1
        except:
            V_freq[w] = 1
            
import operator
sorted_V_freq = sorted(V_freq.items(), key=operator.itemgetter(1))

In [ ]:
# sorted_V[-500:-490]
# let's arbitrarily trim it to 1000 words (total 14417 actually)
vocab_size = 1000 # also experimented with 500

V_map = {}
V_new = sorted_V_freq[-vocab_size:]
for i,tup in enumerate(V_new):
    V_map[tup[0]] = i
len(V_map)
list(V_map.keys())[0]

### Regression framework

In [ ]:
S = len(specialties)
V = len(V_map)
#size_of_var = 2+2*V+2*S+2*S*V
size_of_var = 2+V+2*S+2*S*V # beta_w no longer needed.
print(1,2,2+S,2+S+V,2+S+V+S,2+S+V+S+S*V,size_of_var)

In [ ]:
# initialize 3d array as 2d array
y = {}
for s in specialties:
    y[s] = {'m':{},'f':{}}

# dict to store (later do log of these values) doctor wise frequencies.
V_beta_w = {}
    
# make set out of text aggregations
for i in range(len(df)):
    words = set()
    entry = df.iloc[i]
    if d[entry.name][0] == '-1':
        continue
    for w in entry['text']:
        words.add(w)
# and fill the y 3d matrix (y_sgw)
    sg = d[entry.name]
    for w in words:
        if w not in V_map.keys():
            continue
        try:
            V_beta_w[w] += 1
        except:
            V_beta_w[w] = 1
        try:
            y[sg[0]][sg[1]][w] += 1
        except:
            y[sg[0]][sg[1]][w] = 1

In [ ]:
type(y['int']['m'])
len(V_beta_w.keys())

In [ ]:
def assign(s,g,w):
    #var = [0] * size_of_var # list of vars.
    var = bitarray(size_of_var)
    var.setall(0)
    
    var[0] = 1
    #var[2+V_map[w]] = 1
    var[2+s_map[s]] = 1
    if g == 'f':
        var[1] = 1 # beta_g
        var[2+S+V_map[w]] = 1 # beta_gw
        var[2+S+V+s_map[s]] = 1 # beta_gs
        var[2+S+V+S+S*V+ (s_map[s]*V) + (V_map[w])] = 1 # beta_sgw
    var[2+S+V+S+ (s_map[s]*V) + (V_map[w]) ] = 1 # beta_sw
    return var

In [ ]:
from bitarray import bitarray

In [ ]:
import numpy as np

size_of_var1 = 1+1+V+S # WARNING: V was used earlier too.
def assign2(s,g,w):
    #var = [0] * size_of_var1 # list of vars.
    var = bitarray(size_of_var1)
    var.setall(0)
    #var = np.zeros(size_of_var1, dtype=bool)
    #var = np.zeros(size_of_var1, dtype=int)
    var[0] = 1
    var[2+V_map[w]] = 1
    var[2+V+s_map[s]] = 1
    if g == 'f':
        var[1] = 1
    return var

In [ ]:
import math
count = 0
for s in specialties:
    for g in genders:
        for w in list(y[s][g].keys()):
            count+=1
            # take log, discount the log of general word frequency, i.e. no need to train beta_w.
            y[s][g][w] = math.log(y[s][g][w]) - math.log(V_beta_w[w])
count   

In [ ]:
y['gyn']['f']['good']

In [ ]:
count = 0
X,Y = [],[]
for s in specialties:
    for g in genders:
        for w in list(y[s][g].keys()):
            x = assign(s,g,w)
            ysgw = y[s][g][w]
            X.append(x)
            Y.append(ysgw)
            count+=1
            if count%100 == 0:
                print(count)

In [ ]:
import pickle
LR = {'X':X,'Y':Y}
version = 'july23'
with open('LR_'+version+'.pickle','wb') as h:
    pickle.dump(LR,h,protocol=2)

In [ ]:
X = LR['X']
Y = LR['Y']
Y[500]

In [ ]:
S = len(specialties) # 13
V = vocab_size # 1000
len(X) # 11612
2+S+V+S+V*S+((S)*V)-1 # 12525
len(X[0]) # 12526

In [ ]:
# ignore some
ig = [] # the indices to be ignored.
ig.append(2+S-1)
ig.append(2+S+V-1)
#ig.append(2+S+V+S-1) counted in the i=0 next loop.
for i in range(S):
    ig.append(2+S+V+S+(i*V)-1)
# 2+S+V+S+((S-1)*V)-1 counted.

# If you don't want beta_sgw:
for i in range(2+S+V+S+(S-1)*V-1,len(X[0])):
    ig.append(i)
'''for i in range(2+S+V+S+((S-1)*V)-1,2+S+V+S+((S)*V)-1): # starting one shouldnt have -1. just to test.
    ig.append(i)
for i in range(S): # 0 value counts the last one that should have been counted in loop above.
    ig.append(2+S+V+S+V*S+(i*V)-1)
for i in range(2+S+V+S+V*S+((S-1)*V)-1,2+S+V+S+V*S+((S)*V)-1): # starting one shouldnt have -1. just to test.
    ig.append(i)'''
# last element should be last element.

In [ ]:
# now to actually ignore some variables not needed.
import numpy as np
%time X1 = np.array(X)
X1.shape
%time X2 = np.delete(X1,ig,axis=1)
X2.shape

### Learn coefficients.

In [ ]:
from sklearn import linear_model
regr = linear_model.Ridge()

In [ ]:
%time regr.fit(X2,Y)

In [ ]:
#version = 'july23'
#with open('LR_model_'+version+'.pickle','wb') as h:
#    pickle.dump(regr,h,protocol=2)

In [ ]:
import pickle
version = 'july23'
with open('LR_model_'+version+'.pickle','rb') as h:
    regr = pickle.load(h)

In [ ]:
c = regr.coef_
regr.intercept_

In [ ]:
c[1] # FEMALE coefficient. beta_g

In [ ]:
c[2:2+S] # SPECIALTIES coefficients. beta_s

In [ ]:
beta_w = {}
for i,w in enumerate(c[2:2+V]):
    beta_w[V_new[i]] = w
sorted_beta_w = sorted(beta_w.items(), key=operator.itemgetter(1))
sorted_beta_w[0:10]
# almost, appreciate, putting ... most frequent words not otherwise compensated in beta_gw or beta_sw.
# words that are neither gender-specific nor specialty-specific but are frequent nevertheless, appear here.

In [ ]:
beta_gw = {}
for i,w in enumerate(c[2+S-1:2+S+V-2]):
    beta_gw[V_new[i]] = w
sorted_beta_gw = sorted(beta_gw.items(), key=operator.itemgetter(1))

In [ ]:
sorted_beta_gw[0:20]
# him, he's, his... (Closest to 'm')
sorted_beta_gw[-20:]
# ... herself, podiatrist (Closest to 'f')

In [ ]:
beta_gs = {}
for i,w in enumerate(c[2+S+V-2:2+S+V+S-3]):
    beta_gs[specialties[i]] = w
sorted_beta_gs = sorted(beta_gs.items(), key=operator.itemgetter(1))
sorted_beta_gs
# brain, surg, bone ... int, skin, family (Closer to 'm' towards... Closer to 'f')

In [ ]:
beta_sw = {}
start = 2+S+V+S-3 # constant,gender, (skipped) beta_w, beta_s, beta_gw, beta_gs
for i in range(S-1):
    beta_sw_temp = {}
    for j in range(V-1):
        beta_sw_temp[V_new[j]] = c[start + j]
    start += V-1
    beta_sw[specialties[i]] = beta_sw_temp

In [ ]:
sorted_beta_sw = {}
for i in specialties[:-1]:
    sorted_beta_sw[i] = sorted(beta_sw[i].items(), key=operator.itemgetter(1))

In [ ]:
#sorted_beta_sw['skin'][-10:0]
#tuple_word = list(beta_sw['skin'].keys())[0]
#beta_sw['gyn'][tuple_word]
#sorted_beta_sw['skin'][-1]
for i in specialties[:-1]:
    print(" - "+i+":")
    for j in sorted_beta_sw[i][-10:]:
        print(j)
    print('\n')
# Individual specialties' nearest words.